In [1]:
import akshare as ak
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 读取前需先转UTF-8，删除前面不规整的行，设置默认index名字
trade_data_df = pd.read_csv('../output/trade_data.csv', index_col='index')
trade_data_df['日期'] = pd.to_datetime(trade_data_df['日期'])
# trade_data_df

In [3]:
sorted_df = trade_data_df.sort_values(['名称', '日期'], ascending=False)
sorted_df.reset_index(inplace=True, drop=True)
sorted_df.to_csv('../output/trade_data.csv', index_label='index', mode='w')
# sorted_df

In [4]:

# 设定参数
stock_code = '700' # 股票代码
curr_price = 354.4 # 当前价格
data_range = {'start':'20220101', 'end': '20220630'}
# 设定参数 结束

def select_stocks_by(df, stock_code, data_range):
    selected = df.query('代码 == @stock_code and 日期 >= @data_range["start"] and 日期 <= @data_range["end"]')
    return selected

selected = select_stocks_by(sorted_df, stock_code, data_range)
# selected

## 计算真实投资收益

### 年内投入

In [5]:
analyze_s = pd.Series(data={'股票代码': stock_code, '起始时间': data_range['start'], '截止时间': data_range['end']})

total_invest_2022 = selected['金额'].sum()
analyze_s['累计投入'] = round(total_invest_2022, 2)
# analyze_s['年内投入']

### 年内投入的盈亏

In [6]:
s_df = selected[['名称', '代码', '类型', '日期', '成交价', '数量']].copy()

analyze_s['交易次数'] = s_df['数量'].count()
analyze_s['平均金额'] = round(analyze_s['累计投入'] / analyze_s['交易次数'], 2)

s_df['成本'] = s_df['成交价'] * s_df['数量']
s_df['现价'] = curr_price
s_df['现值'] = s_df['现价'] * s_df['数量']
s_df['盈亏'] = s_df['现值'] - s_df['成本']
profit_and_loss = s_df['盈亏'].sum()
analyze_s['累计收益'] = round(profit_and_loss, 2)
# analyze_s['年内收益率']

### 盈亏比例

In [7]:
margin_2022 = profit_and_loss / total_invest_2022
analyze_s['累计收益率'] = format(margin_2022, '0.2%')

## 计算模拟定投的盈亏

In [8]:
stock_hk_hist_df = ak.stock_hk_hist(symbol='00'+ stock_code, period="weekly", 
                                    start_date=data_range['start'], end_date=data_range['end'], adjust="")
stock_hk_hist_df['日期'] = pd.to_datetime(stock_hk_hist_df['日期'])
stock_hk_hist_df

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2022-01-07,459.8,443.0,463.0,421.0,94718063,4.154575e+10,9.19,-3.02,-13.8,0.99
1,2022-01-14,448.0,473.4,486.0,442.4,96271904,4.499585e+10,9.84,6.86,30.4,1.00
2,2022-01-21,472.0,474.8,478.4,442.0,110066985,5.102685e+10,7.69,0.30,1.4,1.15
3,2022-01-28,470.0,460.0,476.6,455.6,96829780,4.503661e+10,4.42,-3.12,-14.8,1.01
4,2022-02-04,461.4,479.0,483.0,459.0,27360000,1.297358e+10,5.22,4.13,19.0,0.28
5,2022-02-11,482.0,477.0,488.0,469.0,91833786,4.384105e+10,3.97,-0.42,-2.0,0.96
6,2022-02-18,470.2,470.0,482.8,465.4,75917150,3.595034e+10,3.65,-1.47,-7.0,0.79
7,2022-02-25,464.2,424.0,466.8,420.2,156346679,6.874112e+10,9.91,-9.79,-46.0,1.63
8,2022-03-04,420.0,403.2,439.0,400.0,121593142,5.059320e+10,9.20,-4.91,-20.8,1.27
9,2022-03-11,389.0,367.8,398.8,360.0,177847345,6.765363e+10,9.62,-8.78,-35.4,1.85


In [9]:
trade_count = stock_hk_hist_df['日期'].count()
trade_count

26

In [10]:
simulating_df = stock_hk_hist_df[['日期', '收盘']].copy()

shares_to_buy = round(selected['数量'].sum() / trade_count)
simulating_df['数量'] = shares_to_buy
simulating_df['金额'] = simulating_df['收盘'] * simulating_df['数量']
simulating_df['现价'] = curr_price
simulating_df['现值'] = simulating_df['现价'] * simulating_df['数量']
simulating_df['盈亏'] = simulating_df['现值'] - simulating_df['金额']
simulating_df

,日期,收盘,数量,金额,现价,现值,盈亏
0,2022-01-07,443.0,281,124483.0,354.4,99586.4,-24896.6
1,2022-01-14,473.4,281,133025.4,354.4,99586.4,-33439.0
2,2022-01-21,474.8,281,133418.8,354.4,99586.4,-33832.4
3,2022-01-28,460.0,281,129260.0,354.4,99586.4,-29673.6
4,2022-02-04,479.0,281,134599.0,354.4,99586.4,-35012.6
5,2022-02-11,477.0,281,134037.0,354.4,99586.4,-34450.6
6,2022-02-18,470.0,281,132070.0,354.4,99586.4,-32483.6
7,2022-02-25,424.0,281,119144.0,354.4,99586.4,-19557.6
8,2022-03-04,403.2,281,113299.2,354.4,99586.4,-13712.8
9,2022-03-11,367.8,281,103351.8,354.4,99586.4,-3765.4


In [11]:
s_profit_and_loss = simulating_df['盈亏'].sum()
analyze_s['模拟定投收益'] = round(s_profit_and_loss, 2)

### 盈亏比例

In [12]:
s_margin_2022 = s_profit_and_loss / simulating_df['金额'].sum()
analyze_s['模拟定投收益率'] = format(s_margin_2022, '0.2%')

In [13]:
analyze_s

股票代码             700
起始时间        20220101
截止时间        20220630
累计投入       2628566.0
交易次数              18
平均金额       146031.44
累计收益        -41446.0
累计收益率         -1.58%
模拟定投收益     -308257.0
模拟定投收益率      -10.64%
dtype: object